In [10]:
#! /usr/bin/env python3


#################
## Sell Expensive! And Buy Cheap!!
################

import sys
import os
from xcoin_api_client import *
import pprint
import json

import time
from datetime import datetime
# from noti.slack import SlackLogger
from poloniex import *


from yahoo_finance import Currency

coin = 'ETH'

b_api_key = os.environ["BITHUMB_API_KEY"]
b_api_secret = os.environ["BITHUMB_API_SECRET"]
#logger = SlackLogger(os.environ["BITHUMB_API_NAME"])

p_api_key = os.environ["POLONIEX_API_KEY"]
p_api_secret = os.environ["POLONIEX_API_SECRET"]


b_api = XCoinAPI(b_api_key, b_api_secret)
p_api = poloniex(p_api_key, p_api_secret)
############### EMA INITIAL SETUP ##############

amount_dict = {
        "BTC": 0.03,
        "ETH": 0.3,
        "LTC": 3.0,
        "DASH": 0.6,
        "ETC": 3.0,
        "XRP": 5000
        }

threshold_dict = {
        "BTC": 1000,
        "ETH": 200,
        "LTC": 20,
        "DASH": 100,
        "ETC": 20
        }

coin_dict = ["BTC", "ETH", "LTC", "DASH", "ETC", "XRP"]
polo_coin_dict = ["BTC_ETH", "BTC_ETC", "BTC_LTC", "BTC_DASH", "BTC_XRP"]
pform_dict = {'ETH' : '{:12}:\t  BUY : {:10.12f} \t SELL: {:10.12f}',
        'LTC' : '{:12}:\t  BUY : {:10.12f} \t SELL: {:10.12f}',
        'DASH' : '{:12}:\t  BUY : {:10.12f} \t SELL: {:10.12f}',
        'ETC' : '{:12}:\t  BUY : {:10.12f} \t SELL: {:10.12f}',
        'XRP' : '{:12}:\t  BUY : {:10.12f} \t SELL: {:10.12f}'}

#if len(sys.argv) < 2 or (sys.argv[1] not in coin_dict):
#    print("enter valid coin")
#    sys.exit(1)
#coin = sys.argv[1]
#polo_coin = sys.argv[2]

polo_coin = 'BTC_'+coin
pform = pform_dict[coin]

threshold = 0.005 # 0.01
commision = 0.004
# BITHUMB : MAKER/TAKER 0.15 %
# 0.0005 BTC
# 0.01 ETC/ETH/LTC/DASH/XRP
# BITHUMB has no fee with coupon.


# 일반거래
# 레벨    최근 30일간 거래금액    Maker   Taker
# Lv.1    5,000,000원 미만        0.15%   0.15%
# Lv.2    50,000,000원 미만       0.12%   0.13%
# Lv.3    500,000,000원 미만      0.09%   0.11%
# Lv.4    2,500,000,000원 미만    0.06%   0.09%
# Lv.5    10,000,000,000원 미만   0.03%   0.07%
# VIP     50,000,000,000원 미만   0%      0.05%
# VVIP    50,000,000,000원 이상   0%      0%



update_period = 10 # seconds
exchange_count = 0



log_file = "exchange_logs/log.csv"
coin_log_file = "exchange_logs/" + coin + "_log.csv"
amount = amount_dict[coin]

time_ema = 1500/update_period
last_buy_price = 0

b_transParams_BTC = {
        "apiKey" : b_api_key,
        "secretKey" : b_api_secret,
        "currency" : 'BTC',
        "units" : amount/4
        }

b_transParams_coin = {
        "apiKey" : b_api_key,
        "secretKey" : b_api_secret,
        "currency" : coin,
        "units" : amount/4
        }

b_info_orders_Params_BTC = {
        "apiKey" : b_api_key,
        "secretKey" : b_api_secret,
        "currency" : 'BTC'
        }
b_info_orders_Params_coin = {
        "apiKey" : b_api_key,
        "secretKey" : b_api_secret,
        "currency" : coin
        }
p_info_orders_Params = {
        "currencyPair" : polo_coin
        }

my_krw = 0
my_coin = 0
state = False
######################## FUNCTIONS ########################

#################################################
def multiplier(time_period):
    return 2.0 / (time_period + 1)
##################################################


class bithumb_bot:
    def __init__(self):
        self.coin = coin
        self.buy_price = None
        self.sell_price = None
        self.btckrw_buy_price = None
        self.btckrw_sell_price = None
        self.target_buy_price = None
        self.target_sell_price = None

    def collect_price(self):
        btc_orderbook = b_api.xcoinApiCall("/public/orderbook/BTC", {})


        # TODO : BTC amount limit??
        self.btckrw_buy_price = float(btc_orderbook["data"]["asks"][0]["price"])
#         print(self.btckrw_buy_price)
        self.btckrw_sell_price = float(btc_orderbook["data"]["bids"][0]["price"])
#         print(self.btckrw_sell_price)

        target_orderbook = b_api.xcoinApiCall("/public/orderbook/" + coin, {})

        count = 0
        while(float(target_orderbook["data"]["asks"][count]["quantity"])<amount):
            count+=1
#         print(count)
        tarkrw_buy_price = target_orderbook["data"]["asks"][count]["price"]
#         print(tarkrw_buy_price)
        # BTC sell -> TAR buy
        self.tarbtc_buy_price = float(tarkrw_buy_price)/self.btckrw_sell_price


        count = 0
        while(float(target_orderbook["data"]["bids"][count]["quantity"])<amount):
            count+=1
#         print(count)
        tarkrw_sell_price = target_orderbook["data"]["bids"][count]["price"]
        # BTC buy -> TAR sell
        self.tarbtc_sell_price = float(tarkrw_sell_price)/self.btckrw_buy_price
#         print(tarkrw_sell_price)

#         print(self.tarbtc_buy_price)
#         print(self.tarbtc_sell_price)
#         print(target_orderbook)

#         print(pform.format('BITHUMB',self.tarbtc_buy_price, self.tarbtc_sell_price))

# ======= POLO
class poloniex_bot:
    def __init__(self):
        self.buy_price = None
        self.sell_price = None


    def collect_price(self):
        p_price = p_api.returnOrderBook(polo_coin)
        count = 0
        while(float(p_price["asks"][count][1])<amount):
            count+=1

        self.buy_price = float(p_price["asks"][count][0])
        count = 0
        while(float(p_price["bids"][count][1])<amount):
            count+=1

        self.sell_price = float(p_price["bids"][count][0])
#         print(pform.format('POLONIEX',self.buy_price, self.sell_price))
#         print(self.buy_price)
#         print(self.sell_price)



bith_bot = bithumb_bot()
polo_bot = poloniex_bot()

def collect_price(count):
    bith_bot.collect_price()
    polo_bot.collect_price()

def calculate_premium(count):
    global ema
    global ema_grad
    global time_ema
    ##################################################
    closing_price = (polo_bot.sell_price + polo_bot.buy_price)/2
    if(count is 0):
        ema = closing_price
        ema_grad = 0.0
    else:
        ema_grad = (closing_price - ema) * multiplier(time_ema)
        ema = (closing_price - ema) * multiplier(time_ema) + ema
    ######################## PRINT ########################
    print('---------------------------------------------------')
    print()
    print(str(count)+"  [" + coin + "] " + str(datetime.now()))
    # 	print('BITHUMB :  \tBUY: ', b_buy_price, '\tSELL: ', b_sell_price, '\t|')
    # 	print('POLO :   \tBUY: ', p_buy_price, '\tSELL: ', p_sell_price, '\t|')
    print(pform.format('BITHUMB ', bith_bot.tarbtc_buy_price, bith_bot.tarbtc_sell_price))
    print(pform.format('POLONIEX', polo_bot.buy_price, polo_bot.sell_price))
    print("Pemium monitoring: POLO : BTC->Target | BITH : Target->BTC : {:5.2f}".format((bith_bot.tarbtc_sell_price / polo_bot.buy_price - 1)*100))
    print("Pemium monitoring: POLO : Target->BTC | BITH : BTC->Target : {:5.2f}".format((polo_bot.sell_price / bith_bot.tarbtc_buy_price - 1)*100))
    prem = 0
    if(ema_grad>0 and count>10): # TODO default count 60
        if(bith_bot.tarbtc_sell_price > polo_bot.buy_price * (1+commision+threshold)):
            #### POLO : BTC -> Target   /    BITHUMB :  Taret -> BTC
            print('#################### PREMIUM ALERT ####################\a')
            # print('POLONIEX BUY BTC\t / BITHUMB BUY', coin)
            print('POLO : BTC -> Target   /    BITHUMB :  Taret -> BTC')
            print('\tPREM RATIO: ', (bith_bot.tarbtc_sell_price / polo_bot.buy_price - 1)*100, ' %')
            prem = 1
        if(polo_bot.sell_price * (1-commision-threshold) > bith_bot.tarbtc_buy_price): # Each market threshold need comission
            #### POLO : Target -> BTC   /    BITHUMB :  BTC -> Target
            print('#################### PREMIUM ALERT ####################\a')
            print('POLO : Target -> BTC   /    BITHUMB :  BTC -> Target')
            #print('POLONIEX SELL',coin, '\t/ BITHUMB SELL', coin)
            print('\tSELL POLONIEX ', coin, '\tBUY BITHUMB BTC')
            print('\tPREM RATIO: ', (polo_bot.sell_price / bith_bot.tarbtc_buy_price - 1)*100, ' %')
            prem = -1
    print()
    if False and count%10 == 0:
        usdkrw = Currency('USDKRW')
        curr = float(usdkrw.get_ask())
        ret = urlopen(urllib.request.Request('https://api.cryptowat.ch/markets/poloniex/btcusd/price'))
        btcusd = json.loads(ret.read())['result']['price']
        print("BTC premeium KRW/USD : ",str((bith_bot.btckrw_buy_price / (curr * btcusd) )), 'with btcusd =',btcusd )
    return prem

#################################################
def open_order_check():
    global b_api
    global p_api
    status = False
    if(b_api.xcoinApiCall("/info/orders",b_info_orders_Params_coin)["status"] is 5600):
        status = True
    elif(b_api.xcoinApiCall("/info/orders",b_info_orders_Params_BTC)["status"] is 5600):
        status = True

####

class wallet:
    def __init__(coin):
         self.polo_btc = 0.1 # 3,360,000
         self.bith_btc = 0.1 # 3,360,000
         self.bith_krw = 200,000

         self.polo_tar = 1
         self.bith.tar = 1

    def polo_tar_buy_btc_sell(): pass
    def polo_tar_sell_btc_buy(): pass
    def bith_tar_sell():
        pass
    def bith_tar_buy():
        pass
    def bith_btc_sell():
        pass
    def bith_btc_buy():
        pass

    # BITHUMB api slower -> do first
    def bith_sell_polo_buy():
        bith_orderbook()
        bith_tar_sell()
        bith_btc_buy()

        polo_orderbook()
        polo_tar_buy_btc_sell()


    def polo_sell_bith_buy(): #1
        status = bith_orderbook()
        bith_btc_sell()
        bith_tar_buy()

        status = polo_orderbook()
        polo_tar_sell_btc_buy()

    def arbitrage(self, prem_signal):
        if prem_segnal == 1:
            self.polo_sell_bith_buy()
        elif prem_signal == -1:
            self.bith_sell_polo_buy()


#################################################
count = 0
prem_alert = False
time.sleep(update_period)
# count +=1

# TODO EMA visualize
while(True):
    iter_start = time.time()

    collect_price(count)
    if prem_alert: # Prem alerted previously
        wallet.arbitrage(prem_alert)
        prem_alert = 0
    prem_alert = calculate_premium(count)

    iter_duration = time.time() - iter_start
    count+=1
    tsleep = max([update_period-iter_duration, 0])
    time.sleep(tsleep)
    iter_end = time.time()
    print('time step : '+ str(iter_end - iter_start))
    iter_start = iter_end

---------------------------------------------------

0  [ETH] 2017-06-25 19:24:12.259336
BITHUMB     :	  BUY : 0.118383658970 	 SELL: 0.118333826576
POLONIEX    :	  BUY : 0.117808230000 	 SELL: 0.117600070000
Pemium monitoring: POLO : BTC->Target | BITH : Target->BTC :  0.45
Pemium monitoring: POLO : Target->BTC | BITH : BTC->Target : -0.66

time step : 10.00131607055664
---------------------------------------------------

1  [ETH] 2017-06-25 19:24:22.388549
BITHUMB     :	  BUY : 0.118383658970 	 SELL: 0.118333826576
POLONIEX    :	  BUY : 0.117808230000 	 SELL: 0.117600090000
Pemium monitoring: POLO : BTC->Target | BITH : Target->BTC :  0.45
Pemium monitoring: POLO : Target->BTC | BITH : BTC->Target : -0.66

time step : 10.000550270080566
---------------------------------------------------

2  [ETH] 2017-06-25 19:24:30.710183
BITHUMB     :	  BUY : 0.118453791469 	 SELL: 0.118368857312
POLONIEX    :	  BUY : 0.117808230000 	 SELL: 0.117600100000
Pemium monitoring: POLO : BTC->Target | BI

KeyboardInterrupt: 

In [22]:
a = b_api.xcoinApiCall("/public/orderbook/" + 'btc', {})
a

{'data': {'asks': [{'price': '3403000', 'quantity': '1.61857711'},
   {'price': '3404000', 'quantity': '3.55387940'},
   {'price': '3405000', 'quantity': '0.00420000'},
   {'price': '3406000', 'quantity': '0.10000000'},
   {'price': '3408000', 'quantity': '0.99000000'},
   {'price': '3409000', 'quantity': '0.86070000'},
   {'price': '3410000', 'quantity': '5.83687797'},
   {'price': '3411000', 'quantity': '5.77996759'},
   {'price': '3413000', 'quantity': '0.20200000'},
   {'price': '3415000', 'quantity': '2.10000000'},
   {'price': '3417000', 'quantity': '0.04015839'},
   {'price': '3418000', 'quantity': '0.10000000'},
   {'price': '3419000', 'quantity': '6.10620000'},
   {'price': '3420000', 'quantity': '44.93137878'},
   {'price': '3421000', 'quantity': '0.20010000'},
   {'price': '3423000', 'quantity': '1.44738474'},
   {'price': '3424000', 'quantity': '0.60200000'},
   {'price': '3425000', 'quantity': '6.00000000'},
   {'price': '3426000', 'quantity': '1.59966958'},
   {'price': '

In [24]:
p_price = p_api.returnOrderBook(polo_coin)
print(p_sell_price)
print(p_price['asks'])
print()
print(p_buy_price)
print(p_price['bids'])

0.12825
[['0.12809990', 168.64394223], ['0.12810000', 48.07956202], ['0.12821283', 107.30136117], ['0.12821284', 43.35050584], ['0.12823197', 0.00979969], ['0.12830618', 14.44085358], ['0.12835000', 0.7833276], ['0.12839997', 76.224], ['0.12840744', 0.14387681], ['0.12848713', 0.06276887], ['0.12850000', 8.16729192], ['0.12854999', 9.8], ['0.12855000', 0.3], ['0.12857820', 0.00979969], ['0.12864000', 0.3], ['0.12865094', 4.25373916], ['0.12865998', 62.85663811], ['0.12873494', 0.95771869], ['0.12874051', 0.1], ['0.12876743', 0.01001645], ['0.12877733', 0.31061368], ['0.12878060', 0.3921275], ['0.12881897', 0.00099799], ['0.12887998', 91.18332502], ['0.12887999', 107.4988187], ['0.12889389', 0.01096032], ['0.12896932', 0.001], ['0.12899997', 0.01038214], ['0.12899999', 3.67456867], ['0.12900000', 261.00383236], ['0.12900580', 0.0033545], ['0.12906485', 0.00087046], ['0.12907800', 0.22706729], ['0.12907801', 0.16585771], ['0.12908237', 0.63452816], ['0.12910569', 13.59424], ['0.12914330'

In [15]:
a = 14

class bbbb:
    def __init__(self):
        self.hi = a

c = bbbb()

In [ ]:
print('\a')

# Coinone

In [2]:
100 KRW

class wallet:
    def __init__():
        self.polo_btc = 
        self.polo_tar = 1
        self.bith_btc = 1
        self.bith_krw = 1
        self.bith.tar = 1
        
    def polo_tar_buy_btc_sell():
        

        
    def polo_tar_sell_btc_buy()
        
        
    def bith_tar_sell():
        
        
    def bith_tar_buy():
        
    def bith_btc_sell():
        
    def bith_btc_buy():        
        
        
    def bith_sell_polo_buy():
        bith_orderbook()
        bith_tar_sell()
        bith_btc_buy()
        
        polo_orderbook()
        polo_tar_buy_btc_sell()
        
        
    def polo_sell_bith_buy(): #1
        status = polo_orderbook()
        polo_tar_sell_btc_buy()
        
        status = bith_orderbook()
        bith_btc_sell()
        bith_tar_buy()
        


SyntaxError: invalid syntax (<ipython-input-2-ec3b28bab10c>, line 1)

In [8]:
start_time = time.time()
btc_orderbook = b_api.xcoinApiCall("/public/orderbook/BTC", {})
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
target_orderbook = b_api.xcoinApiCall("/public/orderbook/" + coin, {})
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
p_price = p_api.returnOrderBook(polo_coin)
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.1200082302093506 seconds ---
--- 1.1344239711761475 seconds ---
--- 0.28265380859375 seconds ---


In [11]:
btc_orderbook

{'data': {'asks': [{'price': '3355000', 'quantity': '5.93516669'},
   {'price': '3356000', 'quantity': '8.86594658'},
   {'price': '3357000', 'quantity': '6.35381898'},
   {'price': '3359000', 'quantity': '0.25811099'},
   {'price': '3360000', 'quantity': '6.65213537'},
   {'price': '3361000', 'quantity': '4.66447386'},
   {'price': '3363000', 'quantity': '2.97796307'},
   {'price': '3364000', 'quantity': '0.20000000'},
   {'price': '3365000', 'quantity': '2.96838291'},
   {'price': '3366000', 'quantity': '8.71550000'},
   {'price': '3367000', 'quantity': '21.02808866'},
   {'price': '3368000', 'quantity': '0.11551127'},
   {'price': '3369000', 'quantity': '13.38375896'},
   {'price': '3370000', 'quantity': '14.72420029'},
   {'price': '3371000', 'quantity': '3.07127184'},
   {'price': '3373000', 'quantity': '5.50010000'},
   {'price': '3374000', 'quantity': '17.14075000'},
   {'price': '3375000', 'quantity': '3.73755188'},
   {'price': '3376000', 'quantity': '2.35850072'},
   {'price'

In [15]:
p_price['asks'][1]

['0.11838999', 12.382]